In [ ]:
import subprocess
import sys
import streamlit as st
from design import APP_NAME, StyleCSS, StyleColors, display_aladdin_logos_and_app_title

st.set_page_config(
    page_title=APP_NAME,
    page_icon="📄",
    layout="wide",
    initial_sidebar_state="expanded",
)

StyleCSS.set_css_styling()
StyleCSS.set_plotly_template(
    "aladdin",
    list(StyleColors.DATAVIZ_COLORS),
    set_as_default=True,
)
display_aladdin_logos_and_app_title()

@st.cache_resource
def install_packages(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

SETUP_VERSION = "2025-09-local-log"

packages = [
    "certifi",
    "charset-normalizer",
    "faiss-cpu",
    "idna",
    "numpy",
    "packaging",
    "python-dotenv",
    "requests",
    "urllib3",
    "pyarrow",
    "PyPDF2",
    "python-docx",
    "spacy",
]

def ensure_packages() -> None:
    if st.session_state.get("setup_version") == SETUP_VERSION:
        return
    progress_placeholder = st.empty()
    total = len(packages)
    for i, package in enumerate(packages, start=1):
        try:
            install_packages(package)
        except subprocess.CalledProcessError:
            progress_placeholder.empty()
            st.error("Something went wrong while setting things up. Please try again or contact support.")
            return
        percent = int(i / total * 100)
        message = f"Setting up step {i} of {total}..."
        progress_placeholder.info(f"{message} ({percent}% complete)")
    progress_placeholder.success("Setup complete.")
    st.session_state["setup_version"] = SETUP_VERSION
    st.toast("You're all set! Choose 'Upload document' to load an RFP or 'Ask a question' to chat. Provide any required API keys in the sidebar.")

ensure_packages()


In [ ]:
import streamlit as st
import os
import tempfile
import json
import re
import io
import html
import contextlib
from concurrent.futures import ThreadPoolExecutor, as_completed
from cli_streamlit_app import _resolve_concurrency
import time
import random
from datetime import datetime
from uuid import uuid4
from pathlib import Path
from typing import List, Optional, Callable
from cli_app import (
    load_input_text,
    extract_questions,
    build_docx,
)
from qa_core import answer_question, collect_relevant_snippets
from answer_composer import CompletionsClient, get_openai_completion
from input_file_reader.interpreter_sheet import collect_non_empty_cells
from rfp_xlsx_slot_finder import ask_sheet_schema
from rfp_xlsx_apply_answers import write_excel_answers
from rfp_docx_slot_finder import extract_slots_from_docx
from rfp_docx_apply_answers import apply_answers_to_docx
import my_module
from my_module import _classify_intent, _detect_followup, gen_answer
from feedback_storage import build_feedback_store
LOCAL_FEEDBACK_FILE = Path("feedback_log.ndjson")
FEEDBACK_FIELDS = [
    "timestamp",
    "session_id",
    "user_id",
    "feedback_source",
    "feedback_subject",
    "rating",
    "highlights",
    "improvements",
    "comment",
    "question",
    "answer",
    "context_json",
]
CHAT_HIGHLIGHT_OPTIONS = [
    "Accurate and complete",
    "Actionable guidance",
    "Clear explanation",
    "Helpful citations",
    "Fast response",
]
CHAT_IMPROVEMENT_OPTIONS = [
    "Incorrect information",
    "Missing details",
    "Not relevant",
    "Formatting issues",
    "Slow response",
]
DOC_HIGHLIGHT_OPTIONS = [
    "Captured required fields",
    "Accurate answers",
    "Helpful citations",
    "Easy to download",
]
DOC_IMPROVEMENT_OPTIONS = [
    "Incorrect answers",
    "Missing responses",
    "Formatting issues",
    "Download problems",
    "Slow processing",
]
feedback_store = build_feedback_store(FEEDBACK_FIELDS, LOCAL_FEEDBACK_FILE)
def get_current_user() -> str:
    return st.session_state.get("current_user_id", "demo_user")
def serialize_list(items: Optional[List[str]]) -> str:
    if not items:
        return ""
    return " | ".join(item.strip() for item in items if item)
def log_feedback(record: dict) -> None:
    feedback_store.append(record)
def format_context(context: dict) -> str:
    try:
        return json.dumps(context, ensure_ascii=False)
    except Exception:
        return ""
def render_chat_feedback_form(
    message_index: int,
    question: Optional[str],
    answer: str,
    message_payload: dict,
) -> None:
    submitted_map = st.session_state.setdefault("chat_feedback_submitted", {})
    feedback_key = f"chat_{message_index}"
    if submitted_map.get(feedback_key):
        st.caption("Feedback recorded — thank you!")
        return
    with st.expander("How was this answer?", expanded=False):
        rating_key = f"chat_rating_{message_index}"
        rating_choice = st.radio(
            "Overall",
            ["Helpful", "Needs improvement"],
            horizontal=True,
            key=rating_key,
        )
        highlights_key = f"chat_highlights_{message_index}"
        improvements_key = f"chat_improvements_{message_index}"
        with st.form(f"chat_feedback_form_{message_index}"):
            highlights: List[str] = []
            improvements: List[str] = []
            if rating_choice == "Helpful":
                st.session_state.pop(improvements_key, None)
                highlights = st.multiselect(
                    "Highlights",
                    CHAT_HIGHLIGHT_OPTIONS,
                    key=highlights_key,
                )
            else:
                st.session_state.pop(highlights_key, None)
                improvements = st.multiselect(
                    "Opportunities",
                    CHAT_IMPROVEMENT_OPTIONS,
                    key=improvements_key,
                )
            comment = st.text_area(
                "Additional comments",
                placeholder="What should we know?",
                key=f"chat_comment_{message_index}",
            )
            submitted = st.form_submit_button("Submit feedback", use_container_width=True)
            if submitted:
                record = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "session_id": st.session_state.get("session_id", str(uuid4())),
                    "user_id": get_current_user(),
                    "feedback_source": "chat",
                    "feedback_subject": f"assistant_message_{message_index}",
                    "rating": "positive" if rating_choice == "Helpful" else "needs_improvement",
                    "highlights": serialize_list(highlights),
                    "improvements": serialize_list(improvements),
                    "comment": comment.strip(),
                    "question": question or "",
                    "answer": answer,
                    "context_json": format_context(
                        {
                            "chat_history": st.session_state.get("chat_messages", []),
                            "message_index": message_index,
                            "message_payload": message_payload,
                        }
                    ),
                }
                log_feedback(record)
                submitted_map[feedback_key] = True
                st.success("Feedback saved — thank you!")


def _shorten_question_label(text: str) -> str:
    cleaned = (text or '').strip()
    if not cleaned:
        return 'Pending question'
    cleaned = ' '.join(cleaned.split())
    return cleaned[:87] + '...' if len(cleaned) > 90 else cleaned


def _create_live_placeholder(container, idx: int, question_text: str):
    if container is None:
        return None
    label = _shorten_question_label(question_text)
    expander = container.expander(f"Q{idx + 1}: {label}", expanded=False)
    placeholder = expander.empty()
    placeholder.info('Waiting for answer...')
    return placeholder


def _format_live_answer(answer, comments, include_citations: bool) -> str:
    if isinstance(answer, dict):
        ans_text = answer.get('text', '')
    else:
        ans_text = str(answer or '')
    ans_text = ans_text.strip() or '_No answer generated._'
    parts = [f"**Answer:** {ans_text}"]
    if include_citations and comments:
        citation_lines = [f"- {c}" for c in comments if c]
        if citation_lines:
            parts.append('**Citations:**' + ''.join(citation_lines))
    return ''.join(parts)

def render_document_feedback_section(run_context: Optional[dict]) -> None:
    if not run_context:
        return
    submitted = st.session_state.get("doc_feedback_submitted", False)
    expander_label = "Share feedback on this document run"
    with st.expander(expander_label, expanded=not submitted):
        if submitted:
            st.caption("Feedback recorded — thank you!")
            return
        with st.form("document_feedback_form"):
            rating_choice = st.radio(
                "Overall",
                ["Helpful", "Needs improvement"],
                horizontal=True,
            )
            highlights = st.multiselect(
                "What worked well?",
                DOC_HIGHLIGHT_OPTIONS,
            )
            improvements = st.multiselect(
                "What should we improve?",
                DOC_IMPROVEMENT_OPTIONS,
            )
            comment = st.text_area("Additional comments", placeholder="Optional details…")
            submitted_form = st.form_submit_button("Submit feedback", use_container_width=True)
            if submitted_form:
                record = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "session_id": st.session_state.get("session_id", str(uuid4())),
                    "user_id": get_current_user(),
                    "feedback_source": "document",
                    "feedback_subject": run_context.get("uploaded_name", "document_run"),
                    "rating": "positive" if rating_choice == "Helpful" else "needs_improvement",
                    "highlights": serialize_list(highlights),
                    "improvements": serialize_list(improvements),
                    "comment": comment.strip(),
                    "question": "",
                    "answer": "",
                    "context_json": format_context(run_context),
                }
                log_feedback(record)
                st.session_state.doc_feedback_submitted = True
                st.success("Feedback saved — thank you!")
MODEL_DESCRIPTIONS = {
    "gpt-4.1-nano-2025-04-14_research": "Lighter, faster model",
    "o3-2025-04-16_research": "Slower, reasoning model",
}
MODEL_SHORT_NAMES = {
    "gpt-4.1-nano-2025-04-14_research": "4.1",
    "o3-2025-04-16_research": "o3",
}
MODEL_OPTIONS = list(MODEL_DESCRIPTIONS.keys())
FOLLOWUP_DEFAULT_MODEL = "gpt-4.1-nano-2025-04-14_research"
DEFAULT_MODEL = "o3-2025-04-16_research"
DOC_DEFAULT_MODEL = "o3-2025-04-16_research"
try:
    DEFAULT_INDEX = MODEL_OPTIONS.index(DEFAULT_MODEL)
except ValueError:
    DEFAULT_INDEX = 0
    DEFAULT_MODEL = MODEL_OPTIONS[0]
def load_fund_tags() -> List[str]:
    path = Path('~/derivs-tool/rfp-ai-tool/structured_extraction/embedding_data.json').expanduser()
    try:
        with path.open('r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception:
        return []
    tags = {t for item in data for t in item.get('metadata', {}).get('tags', [])}
    return sorted(tags)
class OpenAIClient:
    def __init__(self, model: str):
        self.model = model
    def get_completion(self, prompt: str, json_output: bool = False):
        return get_openai_completion(prompt, self.model, json_output=json_output)
def save_uploaded_file(uploaded_file) -> str:
    suffix = Path(uploaded_file.name).suffix
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=suffix)
    tmp.write(uploaded_file.read())
    tmp.flush()
    return tmp.name
def build_generator(
    search_mode: str,
    fund: Optional[str],
    k: int,
    length: Optional[str],
    approx_words: Optional[int],
    min_confidence: float,
    include_citations: bool,
    llm,
    extra_docs: Optional[List[str]] = None,
):
    def gen(question: str, progress: Optional[Callable[[str], None]] = None):
        ans, cmts = answer_question(
            question,
            search_mode,
            fund,
            k,
            length,
            approx_words,
            min_confidence,
            llm,
            extra_docs=extra_docs,
            progress=progress,
        )
        if not include_citations:
            ans = re.sub(r"\[\d+\]", "", ans)
            return ans
        citations = {
            lbl: {"text": snippet, "source_file": src}
            for lbl, src, snippet, score, date in cmts
        }
        return {"text": ans, "citations": citations}
    return gen
def select_top_preapproved_answers(question: str, hits: List[dict], limit: int = 5) -> List[dict]:
    """Use the Aladdin completions client to pick the most relevant pre-approved answers."""
    if len(hits) <= limit:
        return hits
    formatted = []
    for idx, hit in enumerate(hits, 1):
        snippet = (hit.get("snippet") or "").strip().replace("", " ")
        if len(snippet) > 500:
            snippet = snippet[:497] + "..."
        source = hit.get("source") or "unknown"
        score = hit.get("score")
        if isinstance(score, (int, float)):
            score_repr = f"{score:.3f}"
        else:
            score_repr = str(score) if score is not None else "unknown"
        date = hit.get("date") or "unknown"
        formatted.append(
            f"{idx}. Source: {source}\nScore: {score_repr}\nDate: {date}\nSnippet: {snippet}"
        )
    prompt = (
        "You are ranking pre-approved RFP answers for how well they address a user's question. "
        f"Return a JSON object with a 'selections' array containing up to {limit} items. "
        "Each selection must include an 'index' (1-based) pointing to the candidate and a 'reason' in one or two sentences "
        "explaining how the candidate addresses the user's question."
        f"\n\nQuestion: {question}"
        "\n\nCandidates:\n" + "\n\n".join(formatted)
    )
    model_name = os.environ.get("ALADDIN_RERANK_MODEL", "o3-2025-04-16_research")
    try:
        client = CompletionsClient(model=model_name)
        content, _ = client.get_completion(prompt, json_output=True)
        data = json.loads(content or "{}")
    except Exception as exc:
        print(f"select_top_preapproved_answers failed with {model_name}: {exc}")
        return hits[:limit]
    selected: List[dict] = []
    seen = set()
    def add_hit(position: int, reason: Optional[str] = None) -> None:
        if not isinstance(position, int):
            return
        if not (1 <= position <= len(hits)):
            return
        if position in seen:
            return
        seen.add(position)
        hit_data = dict(hits[position - 1])
        if reason:
            cleaned = " ".join(str(reason).strip().split())
            if cleaned:
                hit_data["selection_reason"] = cleaned
        hit_data.setdefault("selected_by_model", model_name)
        selected.append(hit_data)
    selections = (
        data.get("selections")
        or data.get("choices")
        or data.get("ranked")
        or data.get("results")
        or []
    )
    if isinstance(selections, dict):
        for value in selections.values():
            if isinstance(value, list):
                selections = value
                break
    if isinstance(selections, list):
        for entry in selections:
            if len(selected) == limit:
                break
            reason = None
            idx_value = None
            if isinstance(entry, dict):
                reason = entry.get("reason") or entry.get("rationale") or entry.get("why")
                idx_value = (
                    entry.get("index")
                    or entry.get("idx")
                    or entry.get("rank")
                    or entry.get("position")
                )
            else:
                idx_value = entry
            try:
                pos = int(idx_value)
            except (TypeError, ValueError):
                continue
            add_hit(pos, reason)
    if len(selected) < limit:
        indices = data.get("top_indices") or data.get("top") or data.get("indices") or []
        if isinstance(indices, (list, tuple)):
            for idx in indices:
                if len(selected) == limit:
                    break
                try:
                    pos = int(idx)
                except (TypeError, ValueError):
                    continue
                add_hit(pos, None)
    if len(selected) < limit:
        for position in range(1, len(hits) + 1):
            if len(selected) == limit:
                break
            if position in seen:
                continue
            add_hit(position, None)
    if not selected:
        return hits[:limit]
    return selected[:limit]
def main():
    st.title("RFP Responder")
    if "session_id" not in st.session_state:
        st.session_state.session_id = str(uuid4())
    st.session_state.setdefault("chat_feedback_submitted", {})
    st.session_state.setdefault("doc_feedback_submitted", False)
    st.session_state.setdefault("latest_doc_run", None)
    st.session_state.setdefault("doc_processing_state", "idle")
    st.session_state.setdefault("doc_processing_result", None)
    st.session_state.setdefault("doc_processing_error", None)
    st.markdown(
        """
        <style>
        div.block-container{
            max-width: 900px;
            padding: 32px 48px;
        }
        div[data-testid="stChatMessage"]{
            border-radius: 0;
            border: 1px solid #d4d6db;
            padding: 16px;
            margin-bottom: 16px;
            box-shadow: 0 2px 6px rgba(17, 24, 39, 0.08);
        }
        div[data-testid="stChatMessage-user"]{
            background-color: #f2f5f8;
        }
        div[data-testid="stChatMessage-assistant"]{
            background-color: #ffffff;
        }
        div[data-testid="stChatInput"] textarea{
            border-radius: 0;
            border: 1px solid #d4d6db;
            padding: 12px;
        }
        @keyframes shimmer{
            0%{background-position:-1000px 0;}
            100%{background-position:1000px 0;}
        }
        .shimmer{
            background:linear-gradient(90deg,#d0d0d0 0%,#b0b0b0 50%,#d0d0d0 100%);
            background-size:1000px 100%;
            animation:shimmer 2s infinite linear;
            -webkit-background-clip:text;
            -webkit-text-fill-color:transparent;
        }
        .hit-card{
            border:1px solid #d4d6db;
            border-radius:0;
            padding:18px 22px;
            background-color:#f5f6f8;
            margin-top:18px;
            box-shadow: 0 2px 6px rgba(17, 24, 39, 0.08);
        }
        .hit-meta{
            font-size:0.85rem;
            color:#525760;
            margin-bottom:12px;
        }
        .hit-reason{
            font-size:0.9rem;
            color:#30343b;
            margin-bottom:12px;
        }
        .hit-reason-label{
            font-weight:600;
            color:#1f232a;
            margin-right:6px;
        }
        .hit-snippet{
            font-size:0.95rem;
            line-height:1.6;
            color:#111827;
        }
        .hit-score{
            font-size:0.8rem;
            color:#6b7280;
            margin-top:6px;
        }
        </style>
        """,
        unsafe_allow_html=True,
    )
    view_mode = "User"
    input_mode = st.radio("How would you like to proceed?", ["Upload document", "Ask a question"], index=1, horizontal=True)
    doc_default_model = DOC_DEFAULT_MODEL if DOC_DEFAULT_MODEL in MODEL_OPTIONS else MODEL_OPTIONS[DEFAULT_INDEX]
    llm_model = doc_default_model
    framework_env = os.getenv("ANSWER_FRAMEWORK")
    if framework_env:
        if view_mode == "Developer":
            st.info(f"Using framework from ANSWER_FRAMEWORK: {framework_env}")
        framework = framework_env
    else:
        framework = st.selectbox("Framework", ["aladdin", "openai"], index=0, help="Choose backend for language model.")
    if framework == "aladdin":
        for key, label in [
            ("aladdin_studio_api_key", "Aladdin Studio API key"),
            ("defaultWebServer", "Default Web Server"),
            ("aladdin_user", "Aladdin user"),
            ("aladdin_passwd", "Aladdin password"),
        ]:
            if os.getenv(key):
                if view_mode == "Developer":
                    st.info(f"{key} loaded from environment")
            else:
                val = st.text_input(label, type="password" if "passwd" in key or "api_key" in key else "default")
                if val:
                    os.environ[key] = val
    else:
        if os.getenv("OPENAI_API_KEY"):
            if view_mode == "Developer":
                st.info("OPENAI_API_KEY loaded from environment")
        else:
            api_key = st.text_input("OpenAI API key", type="password", help="API key for OpenAI.")
            if api_key:
                os.environ["OPENAI_API_KEY"] = api_key
    if input_mode == "Upload document":
        uploaded = st.file_uploader(
            "Upload document",
            type=["pdf", "docx", "xls", "xlsx"],
            help="Upload the RFP or question file (PDF, Word, or Excel).",
        )
        if uploaded and Path(uploaded.name).suffix.lower() not in [".pdf", ".docx", ".xls", ".xlsx"]:
            st.warning("Only PDF, Word, or Excel documents are supported.")
            uploaded = None
    else:
        uploaded = None
    if view_mode == "Developer":
        st.info("Search mode fixed to 'both'")
        search_mode = "both"
        fund = st.selectbox(
            "Fund", [""] + load_fund_tags(), index=0,
            help="Filter answers for a specific fund or strategy.",
        )
        llm_model = st.selectbox(
            "LLM model",
            MODEL_OPTIONS,
            index=DEFAULT_INDEX,
            format_func=lambda m: f"{m} - {MODEL_DESCRIPTIONS[m]}",
            help="Model name for generating answers.",
        )
        k_max_hits = st.number_input("Hits per question", value=10, help="Maximum documents retrieved per question.")
        min_confidence = st.number_input("Min confidence", value=0.0, help="Minimum score for retrieved documents.")
        docx_as_text = st.checkbox("Treat DOCX as text", value=False)
        docx_write_mode = st.selectbox("DOCX write mode", ["fill", "replace", "append"], index=0)
        extra_uploads = st.file_uploader(
            "Additional documents",
            type=["pdf", "docx", "xls", "xlsx"],
            accept_multiple_files=True,
            help="Additional PDF or Word documents to include in search.",
        )
        if extra_uploads:
            valid_files = []
            invalid_files = []
            for f in extra_uploads:
                if Path(f.name).suffix.lower() in [".pdf", ".docx", ".xls", ".xlsx"]:
                    valid_files.append(f)
                else:
                    invalid_files.append(f.name)
            if invalid_files:
                st.warning("Unsupported file types were ignored: " + ", ".join(invalid_files))
            extra_uploads = valid_files
    else:
        search_mode = "both"
        fund = st.selectbox(
            "Fund", [""] + load_fund_tags(), index=0,
            help="Select fund or strategy context for better answers.",
        )
        k_max_hits = 20
        min_confidence = 0.0
        docx_as_text = False
        docx_write_mode = "fill"
        extra_uploads = st.file_uploader(
            "Additional documents",
            type=["pdf", "docx", "xls", "xlsx"],
            accept_multiple_files=True,
            help="Additional PDF or Word documents to include in search.",
        )
        if extra_uploads:
            valid_files = []
            invalid_files = []
            for f in extra_uploads:
                if Path(f.name).suffix.lower() in [".pdf", ".docx", ".xls", ".xlsx"]:
                    valid_files.append(f)
                else:
                    invalid_files.append(f.name)
            if invalid_files:
                st.warning("Unsupported file types were ignored: " + ", ".join(invalid_files))
            extra_uploads = valid_files
    with st.expander("More options"):
        if view_mode == "User":
            llm_model = st.selectbox(
                "Model",
                MODEL_OPTIONS,
                index=MODEL_OPTIONS.index(llm_model),
                format_func=lambda m: f"{MODEL_SHORT_NAMES[m]} - {MODEL_DESCRIPTIONS[m]}",
                help="Choose which model generates answers.",
            )
        length_opt = st.selectbox("Answer length", ["auto", "short", "medium", "long"], index=3)
        approx_words = st.text_input("Approx words", value="", help="Approximate words per answer (optional).")
        include_env = os.getenv("RFP_INCLUDE_COMMENTS")
        if include_env is not None:
            include_citations = include_env != "0"
            st.info(f"Using include citations from RFP_INCLUDE_COMMENTS: {include_citations}")
        else:
            include_citations = st.checkbox("Include citations", value=True)
        show_live = st.checkbox("Show questions and answers during processing", value=True)
    if input_mode == "Ask a question":
        extra_docs = [save_uploaded_file(f) for f in extra_uploads] if extra_uploads else None
        llm = CompletionsClient(model=llm_model) if framework == "aladdin" else OpenAIClient(model=llm_model)
        gen = build_generator(
            search_mode,
            fund,
            int(k_max_hits),
            length_opt,
            int(approx_words) if approx_words else None,
            float(min_confidence),
            include_citations,
            llm,
            extra_docs,
        )
        my_module._llm_client = llm
        response_mode = st.radio(
            "Response style",
            ["Generate answer", "Closest pre-approved answers"],
            index=0,
            horizontal=True,
            help="Switch between generating an answer or browsing the closest approved responses.",
        )
        if "chat_messages" not in st.session_state:
            st.session_state.chat_messages = []
        if "question_history" not in st.session_state:
            st.session_state.question_history = []
        sidebar = st.sidebar.container()
        sidebar.markdown("### References")
        answer_idx = 0
        last_user_message = None
        for idx, msg in enumerate(st.session_state.chat_messages):
            with st.chat_message(msg["role"]):
                if msg["role"] == "user":
                    st.markdown(msg.get("content", ""))
                    last_user_message = msg.get("content", "")
                    continue
                if "hits" in msg:
                    st.markdown(msg.get("content", ""))
                    hits = msg.get("hits") or []
                    answer_summary = msg.get("content", "")
                    if hits:
                        summary_lines = []
                        for i, hit in enumerate(hits, 1):
                            snippet = html.escape(hit.get("snippet", ""))
                            source_name = html.escape(hit.get("source", "Unknown"))
                            meta_parts = [f"<strong>{i}. {source_name}</strong>"]
                            score_val = hit.get("score")
                            if isinstance(score_val, (int, float)):
                                meta_parts.append(f"Score {score_val:.3f}")
                            elif score_val:
                                meta_parts.append(f"Score {html.escape(str(score_val))}")
                            date_val = hit.get("date")
                            if date_val:
                                meta_parts.append(html.escape(str(date_val)))
                            meta_line = " · ".join(meta_parts)
                            reason_text = hit.get("selection_reason")
                            reason_block = ""
                            if reason_text:
                                cleaned_reason = " ".join(str(reason_text).strip().split())
                                if cleaned_reason:
                                    reason_block = (
                                        f'<div class="hit-reason"><span class="hit-reason-label">Model rationale:</span>'
                                        f"{html.escape(cleaned_reason)}</div>"
                                    )
                            st.markdown(
                                f"""
                                <div class="hit-card">
                                    <div class="hit-meta">{meta_line}</div>
                                    {reason_block}
                                    <div class="hit-snippet">{snippet}</div>
                                </div>
                                """,
                                unsafe_allow_html=True,
                            )
                            snippet_plain = (hit.get("snippet") or "").strip()
                            if snippet_plain:
                                summary_lines.append(f"{i}. {snippet_plain}")
                    else:
                        st.info(msg.get("empty_message", "No relevant answers found in the approved library."))
                    if hits:
                        joined = "\n".join(summary_lines)
                        if joined:
                            answer_summary = f"{answer_summary}\n{joined}".strip()
                    render_chat_feedback_form(
                        message_index=idx,
                        question=last_user_message,
                        answer=answer_summary,
                        message_payload=msg,
                    )
                    continue
                st.markdown(msg.get("content", ""))
                if msg.get("model"):
                    name = MODEL_SHORT_NAMES.get(msg["model"], msg["model"]) if view_mode == "User" else msg["model"]
                    st.caption(f"Model: {name}")
                if view_mode == "Developer" and msg.get("debug"):
                    st.expander("Debug info").markdown(f"```\n{msg['debug']}\n```")
                if msg["role"] == "assistant" and "hits" not in msg:
                    answer_idx += 1
                    sidebar.markdown(f"**Answer {answer_idx}**")
                    citations_map = msg.get("citations") or {}
                    if citations_map:
                        for lbl, cite in citations_map.items():
                            source_name = (cite.get('source_file') or 'Unknown').strip() or 'Unknown'
                            exp_label = f"[{lbl}] {source_name}"
                            with sidebar.expander(exp_label):
                                section_value = (cite.get('section') or '').strip()
                                if section_value:
                                    st.markdown(f"**Section:** {section_value}**")
                                snippet_text = (cite.get('text') or '').strip()
                                if snippet_text:
                                    st.markdown(snippet_text)
                                else:
                                    st.caption("Snippet not available.")
                    else:
                        sidebar.caption("No source details returned.")
                    render_chat_feedback_form(
                        message_index=idx,
                        question=last_user_message,
                        answer=msg.get("content", ""),
                        message_payload=msg,
                    )
        if prompt := st.chat_input("Ask a question"):
            st.chat_message("user").markdown(prompt)
            st.session_state.chat_messages.append({"role": "user", "content": prompt})
            history = list(st.session_state.get("question_history", []))
            if response_mode == "Closest pre-approved answers":
                with st.chat_message("assistant"):
                    container = st.container()
                    status_placeholder = container.empty()
                    def _format_preapproved_status(raw: str) -> str:
                        text = (raw or "").strip()
                        lower = text.lower()
                        if "search" in lower:
                            return "Stage 1/3 - Searching approved library..."
                        if "candidate" in lower or "filter" in lower:
                            return "Stage 2/3 - Filtering matches..."
                        if any(token in lower for token in ("complete", "finished", "done", "ready")):
                            return "Stage 3/3 - Finalizing results..."
                        return text or "Working..."

                    def _set_preapproved_status(message: str, final: bool = False) -> None:
                        display = message or "Working..."
                        if final:
                            status_placeholder.success(display)
                        else:
                            status_placeholder.markdown(
                                f'<span class="shimmer">{display}</span>',
                                unsafe_allow_html=True,
                            )

                    def update_status(msg: str) -> None:
                        formatted = _format_preapproved_status(msg)
                        lower = (msg or "").lower()
                        final = any(token in lower for token in ("complete", "finished", "done", "ready"))
                        _set_preapproved_status(formatted, final=final)

                    _set_preapproved_status("Stage 1/3 - Searching approved library...")
                    rows = collect_relevant_snippets(
                        q=prompt,
                        mode=search_mode,
                        fund=fund,
                        k=int(k_max_hits),
                        min_confidence=float(min_confidence),
                        llm=llm,
                        extra_docs=extra_docs,
                        progress=update_status,
                    )
                    _set_preapproved_status("Stage 3/3 - Results ready.", final=True)
                    hits_payload = []
                    if rows:
                        for i, (lbl, src_name, snippet, score, date_str) in enumerate(rows, 1):
                            hits_payload.append(
                                {
                                    "label": lbl,
                                    "source": src_name,
                                    "snippet": snippet,
                                    "score": score,
                                    "date": date_str,
                                    "original_index": i,
                                }
                            )
                        hits_to_show = select_top_preapproved_answers(prompt, hits_payload)
                        container.markdown("**Closest pre-approved answers**")
                        model_used = os.environ.get("ALADDIN_RERANK_MODEL", "o3-2025-04-16_research")
                        summary_lines = []
                        for display_idx, hit in enumerate(hits_to_show, 1):
                            snippet_html = html.escape(hit.get("snippet", ""))
                            snippet_plain = (hit.get("snippet") or "").strip()
                            if snippet_plain:
                                summary_lines.append(f"{display_idx}. {snippet_plain}")
                            source_html = html.escape(hit.get("source") or "Unknown")
                            meta_parts = [f"<strong>{display_idx}. {source_html}</strong>"]
                            score = hit.get("score")
                            if isinstance(score, (int, float)):
                                meta_parts.append(f"Score {score:.3f}")
                            elif score:
                                meta_parts.append(f"Score {html.escape(str(score))}")
                            date_str = hit.get("date")
                            if date_str:
                                meta_parts.append(html.escape(str(date_str)))
                            original_idx = hit.get("original_index")
                            if original_idx and original_idx != display_idx:
                                meta_parts.append(f"Orig #{original_idx}")
                            meta_line = " · ".join(meta_parts)
                            hit["rank"] = display_idx
                            hit.setdefault("selected_by_model", model_used)
                            reason_text = hit.get("selection_reason")
                            reason_block = ""
                            if reason_text:
                                cleaned_reason = " ".join(str(reason_text).strip().split())
                                if cleaned_reason:
                                    reason_block = (
                                        f'<div class="hit-reason"><span class="hit-reason-label">Model rationale:</span>'
                                        f"{html.escape(cleaned_reason)}</div>"
                                    )
                            container.markdown(
                                f"""
                                <div class="hit-card">
                                    <div class="hit-meta">{meta_line}</div>
                                    {reason_block}
                                    <div class="hit-snippet">{snippet_html}</div>
                                </div>
                                """,
                                unsafe_allow_html=True,
                            )
                        msg = {"role": "assistant", "content": "**Closest pre-approved answers**", "hits": hits_to_show}
                        answer_summary = msg.get("content", "")
                        if summary_lines:
                            answer_summary = f"{answer_summary}\n" + "\n".join(summary_lines)
                        answer_summary = answer_summary.strip()
                        new_index = len(st.session_state.chat_messages)
                        render_chat_feedback_form(
                            message_index=new_index,
                            question=prompt,
                            answer=answer_summary,
                            message_payload=msg,
                        )
                    else:
                        empty_message = "No relevant answers found in the approved library."
                        container.info(empty_message)
                        msg = {
                            "role": "assistant",
                            "content": "Closest pre-approved answers",
                            "hits": [],
                            "empty_message": empty_message,
                        }
                        new_index = len(st.session_state.chat_messages)
                        render_chat_feedback_form(
                            message_index=new_index,
                            question=prompt,
                            answer=empty_message,
                            message_payload=msg,
                        )
            else:
                with st.chat_message("assistant"):
                    status_placeholder = st.empty()
                    message_placeholder = st.empty()

                    def _format_answer_status(raw: str) -> str:
                        text = (raw or "").strip()
                        lower = text.lower()
                        if "context" in lower or "history" in lower:
                            return "Stage 1/4 - Analyzing conversation context..."
                        if "search" in lower:
                            return "Stage 2/4 - Searching knowledge base..."
                        if "candidate" in lower or "filter" in lower:
                            return "Stage 3/4 - Filtering snippets..."
                        if "generating" in lower:
                            return "Stage 4/4 - Generating answer..."
                        if any(token in lower for token in ("complete", "finished", "done", "ready")):
                            return "Stage 4/4 - Answer ready."
                        return text or "Working..."

                    def _set_answer_status(message: str, final: bool = False) -> None:
                        display = message or "Working..."
                        if final:
                            status_placeholder.success(display)
                        else:
                            status_placeholder.markdown(
                                f'<span class="shimmer">{display}</span>',
                                unsafe_allow_html=True,
                            )

                    def update_status(msg: str) -> None:
                        formatted = _format_answer_status(msg)
                        lower = (msg or "").lower()
                        final = any(token in lower for token in ("complete", "finished", "done", "ready"))
                        _set_answer_status(formatted, final=final)

                    _set_answer_status("Stage 1/4 - Analyzing conversation context...")
                    intent = _classify_intent(prompt, history)
                    follow = _detect_followup(prompt, history) if intent == "follow_up" else []
                    buf = io.StringIO() if view_mode == "Developer" else None
                    response_model = llm_model
                    restore_client = None
                    call_fn = gen_answer if intent == "follow_up" else gen
                    try:
                        if intent == "follow_up" and view_mode != "Developer":
                            response_model = FOLLOWUP_DEFAULT_MODEL
                            followup_llm = (
                                llm
                                if response_model == llm_model
                                else (
                                    CompletionsClient(model=response_model)
                                    if framework == "aladdin"
                                    else OpenAIClient(model=response_model)
                                )
                            )
                            restore_client = my_module._llm_client
                            my_module._llm_client = followup_llm
                        if buf:
                            with contextlib.redirect_stdout(buf):
                                ans = call_fn(prompt, progress=update_status)
                        else:
                            ans = call_fn(prompt, progress=update_status)
                    finally:
                        if restore_client is not None:
                            my_module._llm_client = restore_client
                    _set_answer_status("Stage 4/4 - Answer ready.", final=True)
                    debug_text = (
                        f"Intent: {intent}",
                        f"Follow-up indices: {follow}",
                        f"{buf.getvalue()}"
                        if buf
                        else ""
                    )
                    text = ans.get("text", "") if isinstance(ans, dict) else ans
                    citations = ans.get("citations", {}) if isinstance(ans, dict) else {}
                    message_placeholder.markdown(text)
                    label = MODEL_SHORT_NAMES.get(response_model, response_model) if view_mode == "User" else response_model
                    st.caption(f"Model: {label}")
                    if view_mode == "Developer":
                        st.expander("Debug info").markdown(f"```\n{debug_text}\n```")
                    if intent != "follow_up":
                        my_module.QUESTION_HISTORY.append(prompt)
                        my_module.QA_HISTORY.append({"question": prompt, "answer": text, "citations": []})
                    payload = {"role": "assistant", "content": text, "citations": citations, "model": response_model}
                    side_idx = sum(
                        1 for m in st.session_state.get("chat_messages", []) if m.get("role") == "assistant"
                    ) + 1
                    sidebar.markdown(f"**Answer {side_idx}**")
                    if citations:
                        for lbl, cite in citations.items():
                            source_name = (cite.get('source_file') or 'Unknown').strip() or 'Unknown'
                            with sidebar.expander(f"[{lbl}] {source_name}"):
                                section_value = (cite.get('section') or '').strip()
                                if section_value:
                                    st.markdown(f"**Section:** {section_value}**")
                                snippet_text = (cite.get('text') or '').strip()
                                st.markdown(snippet_text if snippet_text else "_Snippet not available._")
                    else:
                        sidebar.caption("No source details returned.")
                    new_index = len(st.session_state.chat_messages)
                    render_chat_feedback_form(
                        message_index=new_index,
                        question=prompt,
                        answer=text,
                        message_payload=payload,
                    )
                msg = payload
                if view_mode == "Developer":
                    msg["debug"] = debug_text
            st.session_state.chat_messages.append(msg)
            history.append(prompt)
            st.session_state.question_history = history
    else:
        run_clicked = st.button("Run")
        processing_state = st.session_state.get("doc_processing_state", "idle")
        processing_result = st.session_state.get("doc_processing_result")
        processing_error = st.session_state.get("doc_processing_error")
        if processing_state == "started" and not run_clicked:
            st.info("Document processing is in progress. Please wait while the run completes.")
        if processing_state == "error" and processing_error and not run_clicked:
            st.error(f"Document processing failed: {processing_error}")
            st.session_state.doc_processing_state = "idle"
            st.session_state.doc_processing_error = None
            st.session_state.doc_processing_result = None
        if processing_state == "finished" and processing_result is not None and not run_clicked:
            st.success("Document processing completed. You can download the results below or start another run.")
        if run_clicked and processing_state == "started":
            st.warning("A document run is already in progress. Please wait for it to finish.")
            st.stop()
        if run_clicked and uploaded is not None and fund:
            st.session_state.doc_processing_state = "started"
            st.session_state.doc_processing_result = None
            st.session_state.doc_processing_error = None
            st.session_state.doc_processing_started_at = datetime.utcnow().isoformat()
            st.session_state.doc_feedback_submitted = False
            st.session_state.latest_doc_run = None
            run_context = None
            phase_placeholder = st.empty()
            sub_placeholder = st.empty()
            dev_placeholder = st.empty()
            dev_logs = []
            state = {"step": 0, "phase": None}
            suffix = Path(uploaded.name).suffix.lower()
            base_steps = 1
            if suffix in (".xlsx", ".xls"):
                branch_steps = 4
            elif suffix == ".docx" and not docx_as_text:
                branch_steps = 3
            else:
                branch_steps = 3
            total_steps = base_steps + branch_steps + 1
            step_bar = st.progress(0)
            def log_step(dev_msg, user_msg=None):
                if user_msg and user_msg != state["phase"]:
                    state["phase"] = user_msg
                    phase_placeholder.markdown(f"**{state['phase']}**")
                    sub_placeholder.empty()
                sub_placeholder.markdown(dev_msg)
                if view_mode == "Developer":
                    dev_logs.append(f"{state['phase']}: {dev_msg}")
                    dev_placeholder.markdown("\n".join(f"{i+1}. {m}" for i, m in enumerate(dev_logs)))
                state["step"] += 1
                step_bar.progress(state["step"] / total_steps, text=state["phase"])
            try:
                log_step("Saving uploaded file", "Preparing document...")
                input_path = save_uploaded_file(uploaded)
                extra_docs = [save_uploaded_file(f) for f in extra_uploads] if extra_uploads else None
                llm = CompletionsClient(model=llm_model) if framework == "aladdin" else OpenAIClient(model=llm_model)
                if suffix in (".xlsx", ".xls"):
                    log_step("Collecting non-empty cells", "Reading workbook...")
                    _ = collect_non_empty_cells(input_path)
                    log_step("Inferring sheet schema", "Analyzing workbook structure...")
                    schema = ask_sheet_schema(input_path)
                    log_step("Building answer generator", "Preparing answer generator...")
                    gen = build_generator(
                        search_mode,
                        fund,
                        int(k_max_hits),
                        length_opt,
                        int(approx_words) if approx_words else None,
                        float(min_confidence),
                        include_citations,
                        llm,
                        extra_docs,
                    )
                    log_step("Starting question-answering", "Generating answers...")
                    total_qs = len(schema)
                    if total_qs == 0:
                        st.info("No questions detected in this workbook.")
                        answers = []
                    else:
                        answers = [None] * total_qs
                        worker_limit = _resolve_concurrency(None)
                        worker_limit = max(1, min(worker_limit, total_qs))
                        progress_container = st.container()
                        qa_box = st.container() if show_live else None
                        placeholders = [None] * total_qs
                        progress = progress_container.progress(0.0)
                        futures = {}
                        with ThreadPoolExecutor(max_workers=worker_limit) as pool:
                            for idx, entry in enumerate(schema):
                                question = (entry.get("question_text") or "").strip()
                                if show_live and qa_box is not None and placeholders[idx] is None:
                                    placeholders[idx] = _create_live_placeholder(qa_box, idx, question)
                                futures[pool.submit(gen, question)] = (idx, question)
                            completed = 0
                            for fut in as_completed(futures):
                                idx, question = futures[fut]
                                ans = fut.result()
                                answers[idx] = ans
                                completed += 1
                                progress.progress(completed / total_qs, text=f"{completed}/{total_qs}")
                                if show_live and placeholders[idx] is not None:
                                    placeholders[idx].markdown(_format_live_answer(ans, [], include_citations))
                    
                    out_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx")
                    write_excel_answers(
                        schema,
                        answers,
                        input_path,
                        out_tmp.name,
                        include_comments=include_citations,
                    )
                    with open(out_tmp.name, "rb") as f:
                        st.download_button(
                            "Download answered workbook",
                            f,
                            file_name=Path(uploaded.name).stem + "_answered.xlsx",
                        )
                    if include_citations:
                        base, _ = os.path.splitext(out_tmp.name)
                        comments_path = base + "_comments.docx"
                        if os.path.exists(comments_path):
                            with open(comments_path, "rb") as f:
                                st.download_button(
                                    "Download comments DOCX",
                                    f,
                                    file_name=Path(uploaded.name).stem + "_comments.docx",
                                )
                    qa_pairs = []
                    for entry, ans in zip(schema, answers):
                        question_text = (entry.get("question_text") or "").strip()
                        answer_text = ans.get("text", "") if isinstance(ans, dict) else ans
                        qa_pairs.append({"question": question_text, "answer": answer_text})
                    run_context = {
                        "mode": "excel",
                        "uploaded_name": uploaded.name,
                        "fund": fund,
                        "search_mode": search_mode,
                        "include_citations": include_citations,
                        "length": length_opt,
                        "approx_words": approx_words,
                        "extra_documents": [f.name for f in extra_uploads] if extra_uploads else [],
                        "qa_pairs": qa_pairs,
                        "schema": schema,
                        "timestamp": datetime.utcnow().isoformat(),
                    }
                elif suffix == ".docx" and not docx_as_text:
                    log_step("Extracting slots from DOCX", "Analyzing document...")
                    slots_payload = extract_slots_from_docx(input_path)
                    slots_tmp = tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False, suffix=".json")
                    json.dump(slots_payload, slots_tmp)
                    slots_tmp.flush()
                    log_step("Building answer generator", "Preparing answer generator...")
                    gen = build_generator(
                        search_mode,
                        fund,
                        int(k_max_hits),
                        length_opt,
                        int(approx_words) if approx_words else None,
                        float(min_confidence),
                        include_citations,
                        llm,
                        extra_docs,
                    )
                    log_step("Starting question-answering", "Generating answers...")
                    slot_list = slots_payload.get('slots', [])
                    skipped_entries = slots_payload.get('skipped_slots', [])
                    heuristic_entries = slots_payload.get('heuristic_skips', [])
                    combined_skips = []
                    if skipped_entries:
                        for entry in skipped_entries:
                            combined_skips.append({
                                'question_text': (entry.get('question_text') or '').strip() or '[blank question text]',
                                'reason_key': entry.get('reason') or 'unspecified',
                                'source': 'slot_filter',
                            })
                    if heuristic_entries:
                        for entry in heuristic_entries:
                            combined_skips.append({
                                'question_text': (entry.get('question_text') or '').strip() or '[blank question text]',
                                'reason': entry.get('reason', 'unspecified'),
                                'source': 'heuristic',
                            })
                    if combined_skips:
                        reason_labels = {
                            'table_reference': 'mentions a table; tables are not supported yet',
                            'blank_question_text': 'blank question text',
                            'heuristic_veto': 'failed question heuristics',
                            'llm_veto': 'LLM candidate rejected by heuristics',
                        }
                        st.warning(f"Skipped {len(combined_skips)} question(s) that cannot be answered automatically.")
                        with st.expander('View skipped questions', expanded=False):
                            for entry in combined_skips:
                                if entry['source'] == 'slot_filter':
                                    reason_label = reason_labels.get(entry['reason_key'], entry['reason_key'] or 'unspecified')
                                else:
                                    reason_label = entry.get('reason', 'unspecified')
                                st.markdown(f"- **{entry['question_text']}** — {reason_label}")

                    num_q = len(slot_list)
                    answers_dict = {}
                    if num_q == 0:
                        st.info("No questions detected in this document.")
                    else:
                        max_workers = _resolve_concurrency(None)
                        max_workers = max(1, min(max_workers, num_q))

                        def _make_client():
                            return CompletionsClient(model=llm_model) if framework == "aladdin" else OpenAIClient(model=llm_model)

                        clients = [_make_client() for _ in range(max_workers)]
                        gens = [
                            build_generator(
                                search_mode,
                                fund,
                                int(k_max_hits),
                                length_opt,
                                int(approx_words) if approx_words else None,
                                float(min_confidence),
                                include_citations,
                                client,
                                extra_docs,
                            )
                            for client in clients
                        ]

                        placeholders = [None] * num_q
                        progress_container = st.container()
                        qa_box = st.container() if show_live else None
                        if show_live and qa_box is not None:
                            for idx, slot in enumerate(slot_list):
                                q_text = (slot.get("question_text") or "").strip()
                                placeholders[idx] = _create_live_placeholder(qa_box, idx, q_text)

                        progress = progress_container.progress(0.0)

                        def _safe_generate(worker_id: int, i: int, slot: dict):
                            gen_fn = gens[worker_id]
                            q_text = (slot.get("question_text") or "").strip()
                            attempts = int(os.getenv("RFP_QA_RETRIES", "3"))
                            base_sleep = float(os.getenv("RFP_QA_BACKOFF", "1.5"))
                            for attempt in range(attempts):
                                try:
                                    result = gen_fn(q_text)
                                    return (i, slot.get("id", f"slot_{i}"), q_text, result, None)
                                except Exception as exc:
                                    if attempt + 1 == attempts:
                                        return (i, slot.get("id", f"slot_{i}"), q_text, None, exc)
                                    time.sleep((base_sleep ** attempt) + random.random() * 0.35)

                        with ThreadPoolExecutor(max_workers=max_workers) as ex:
                            futures = []
                            for i, slot in enumerate(slot_list, 1):
                                worker_id = (i - 1) % max_workers
                                futures.append(ex.submit(_safe_generate, worker_id, i, slot))

                            completed = 0
                            for fut in as_completed(futures):
                                i, slot_id, q_text, ans, err = fut.result()
                                completed += 1
                                progress.progress(completed / num_q, text=f"{completed}/{num_q}")

                                if err is not None:
                                    err_msg = f"[temporary error] Could not generate answer. ({err})"
                                    answers_dict[slot_id] = {"text": err_msg} if include_citations else err_msg
                                    if show_live and placeholders[i - 1] is not None:
                                        err_payload = {"text": err_msg} if include_citations else err_msg
                                        placeholders[i - 1].markdown(_format_live_answer(err_payload, [], include_citations))
                                    continue

                                answers_dict[slot_id] = ans
                                if show_live and placeholders[i - 1] is not None:
                                    comments = []
                                    if isinstance(ans, dict):
                                        raw_comments = ans.get('citations') or ans.get('comments') or []
                                        if isinstance(raw_comments, list):
                                            comments = raw_comments
                                        elif raw_comments:
                                            comments = [raw_comments]
                                    placeholders[i - 1].markdown(_format_live_answer(ans, comments, include_citations))

                    answers_tmp = tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False, suffix=".json")
                    json.dump({'by_id': answers_dict}, answers_tmp)
                    answers_tmp.flush()
                    out_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
                    apply_answers_to_docx(
                        docx_path=input_path,
                        slots_json_path=slots_tmp.name,
                        answers_json_path=answers_tmp.name,
                        out_path=out_tmp.name,
                        mode=docx_write_mode,
                        generator=None,
                        gen_name="streamlit_app:rag_gen",
                    )
                    with open(out_tmp.name, "rb") as f:
                        st.download_button(
                            "Download answered DOCX",
                            f,
                            file_name=Path(uploaded.name).stem + "_answered.docx",
                        )
                    qa_pairs = []
                    slot_list = slots_payload.get('slots', [])
                    for slot in slot_list:
                        question_text = (slot.get("question_text") or "").strip()
                        answer_obj = answers_dict.get(slot.get('id'))
                        answer_text = ""
                        if isinstance(answer_obj, dict):
                            answer_text = answer_obj.get("text", "")
                        elif isinstance(answer_obj, str):
                            answer_text = answer_obj
                        qa_pairs.append({"question": question_text, "answer": answer_text})
                    run_context = {
                        "mode": "docx_slots",
                        "uploaded_name": uploaded.name,
                        "fund": fund,
                        "search_mode": search_mode,
                        "include_citations": include_citations,
                        "docx_write_mode": docx_write_mode,
                        "extra_documents": [f.name for f in extra_uploads] if extra_uploads else [],
                        "qa_pairs": qa_pairs,
                        "slots": slots_payload,
                        "timestamp": datetime.utcnow().isoformat(),
                    }
                else:
                    log_step("Loading input text", "Reading document...")
                    raw = load_input_text(input_path)
                    log_step("Extracting questions", "Finding questions...")
                    questions = extract_questions(raw, llm)
                    log_step("Starting question-answering", "Generating answers...")
                    total_qs = len(questions)
                    if total_qs == 0:
                        st.info("No questions could be extracted from the document.")
                        answers = []
                        comments = []
                    else:
                        answers = [None] * total_qs
                        comments = [None] * total_qs
                        worker_limit = _resolve_concurrency(None)
                        worker_limit = max(1, min(worker_limit, total_qs))
                        progress_container = st.container()
                        qa_box = st.container() if show_live else None
                        placeholders = [None] * total_qs
                        progress = progress_container.progress(0.0)

                        def _generate_text_answer(question_text: str):
                            return answer_question(
                                question_text,
                                search_mode,
                                fund,
                                int(k_max_hits),
                                length_opt,
                                int(approx_words) if approx_words else None,
                                float(min_confidence),
                                llm,
                            )

                        futures = {}
                        with ThreadPoolExecutor(max_workers=worker_limit) as pool:
                            for idx, q in enumerate(questions):
                                if show_live and qa_box is not None and placeholders[idx] is None:
                                    placeholders[idx] = _create_live_placeholder(qa_box, idx, q)
                                futures[pool.submit(_generate_text_answer, q)] = idx
                            completed = 0
                            for fut in as_completed(futures):
                                idx = futures[fut]
                                ans, cmts = fut.result()
                                if not include_citations:
                                    ans = re.sub(r"\\[\\d+\\]", "", ans)
                                    cmts = []
                                answers[idx] = ans
                                comments[idx] = cmts
                                completed += 1
                                progress.progress(completed / total_qs, text=f"{completed}/{total_qs}")
                                if show_live and placeholders[idx] is not None:
                                    placeholders[idx].markdown(_format_live_answer(ans, cmts, include_citations))
                    
                    qa_doc = build_docx(
                        questions,
                        answers,
                        comments,
                        include_comments=include_citations,
                    )
                    out_tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
                    out_tmp.write(qa_doc)
                    out_tmp.flush()
                    with open(out_tmp.name, "rb") as f:
                        st.download_button(
                            "Download Q/A report",
                            f,
                            file_name=Path(uploaded.name).stem + "_answered.docx",
                        )
                    qa_pairs = []
                    for question, answer_text in zip(questions, answers):
                        rendered_answer = answer_text.get("text", "") if isinstance(answer_text, dict) else answer_text
                        qa_pairs.append({"question": question, "answer": rendered_answer})
                    run_context = {
                        "mode": "document_summary",
                        "uploaded_name": uploaded.name,
                        "fund": fund,
                        "search_mode": search_mode,
                        "include_citations": include_citations,
                        "length": length_opt,
                        "approx_words": approx_words,
                        "extra_documents": [f.name for f in extra_uploads] if extra_uploads else [],
                        "qa_pairs": qa_pairs,
                        "timestamp": datetime.utcnow().isoformat(),
                    }
                step_bar.progress(1.0, text="Done")
                st.session_state.doc_processing_state = "finished"
                st.session_state.doc_processing_finished_at = datetime.utcnow().isoformat()
                st.session_state.doc_processing_result = run_context
                if run_context:
                    st.session_state.latest_doc_run = run_context
                    render_document_feedback_section(run_context)
            except Exception as exc:
                st.session_state.doc_processing_state = "error"
                st.session_state.doc_processing_error = str(exc)
                st.error("Something went wrong while processing the document. Please try again.")
                st.exception(exc)
                st.stop()
        elif run_clicked and not fund:
            st.warning("Please select a fund or strategy before running.")
        elif run_clicked:
            st.warning("Please upload a document before running.")
        elif st.session_state.get("latest_doc_run"):
            render_document_feedback_section(st.session_state.get("latest_doc_run"))
    
    
if __name__ == "__main__":
    main()
